In [1]:
import spacy
nlp=spacy.load("en_core_web_sm")
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix

In [5]:
data=pd.read_csv("data/train/news.csv")

In [4]:
data.shape

(6335, 4)

In [6]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
data.isna().sum()
data.dropna(inplace=True)

In [7]:
data.drop_duplicates(inplace=True)

In [8]:
data=data.drop(['Unnamed: 0','text'],axis=1)

In [10]:
data.head()

,title,label
0,You Can Smell Hillary’s Fear,FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,FAKE
2,Kerry to go to Paris in gesture of sympathy,REAL
3,Bernie supporters on Twitter erupt in anger ag...,FAKE
4,The Battle of New York: Why This Primary Matters,REAL


In [41]:
pattern = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
ps=PorterStemmer()

In [45]:
corpus=[]
for i in range(0,data.shape[0]):
    sent=re.sub(pattern,' ',data.title[i])
    sent=sent.lower()
    words=nltk.word_tokenize(sent)
    words=[word for word in words if word not in set(stopwords.words('english'))]
    words=[ps.stem(word) for word in words]
    sent=' '.join(words)
    corpus.append(sent)

In [46]:
corpus[0:5]

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter']

In [56]:
data['corpus']=corpus

In [57]:
data.head()

,title,label,corpus
0,You Can Smell Hillary’s Fear,FAKE,smell hillari fear
1,Watch The Exact Moment Paul Ryan Committed Pol...,FAKE,watch exact moment paul ryan commit polit suic...
2,Kerry to go to Paris in gesture of sympathy,REAL,kerri go pari gestur sympathi
3,Bernie supporters on Twitter erupt in anger ag...,FAKE,berni support twitter erupt anger dnc tri warn
4,The Battle of New York: Why This Primary Matters,REAL,battl new york primari matter


In [67]:
X=data['corpus']
y=data['label']

In [68]:
cv=CountVectorizer()
X=cv.fit_transform(X).toarray()

In [73]:
X.shape

(6335, 7113)

In [74]:
X[0:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Model Building

In [76]:
from sklearn.model_selection import train_test_split


In [78]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=0)

In [79]:
classifier=MultinomialNB()

In [80]:
classifier.fit(X_train,y_train)

MultinomialNB()

In [81]:
classifier.score(X_train,y_train)

0.9230942715381145

In [82]:
y_pred=classifier.predict(X_test)

In [ ]:
acc

In [84]:
accuracy_score(y_test,y_pred)

0.8100999473961074

In [87]:
confusion_matrix(y_test,y_pred)

array([[703, 230],
       [131, 837]], dtype=int64)

In [91]:
import pickle

In [92]:
pickle.dump(classifier,open("model/model.pkl","wb"))

In [93]:
model=pickle.load(open("model/model.pkl","rb"))

In [94]:
prdict=model.predict(X_test)

In [95]:
accuracy_score(y_test,y_pred)

0.8100999473961074

In [127]:
import nltk
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

pattern = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

def preProcessTest(news):
    ps=PorterStemmer()
    news=re.sub(pattern,' ',news)
    news=news.lower()
    words=nltk.word_tokenize(news)
    words=[word for word in words if word not in set(stopwords.words('english'))]
    words=[ps.stem(word) for word in words]
    news=' '.join(words)
    
    return news

def preProcessTrain():
    data=pd.read_csv("data/train/news.csv")
    data.dropna(inplace=True)
    data=data.drop(['Unnamed: 0','text'],axis=1)
    data.reset_index(inplace=True)
    
    corpus=[]
    for i in range(0,data.shape[0]):
        sent=re.sub(pattern,' ',data.title[i])
        sent=sent.lower()
        words=nltk.word_tokenize(sent)
        words=[word for word in words if word not in set(stopwords.words('english'))]
        words=[ps.stem(word) for word in words]
        sent=' '.join(words)
        corpus.append(sent)
    y=data['label']
    
    return corpus,y

  `
    

In [128]:
X,y=preProcessTrain()

In [ ]:
from com_in_nitin_utils.utils import preProcessTrain
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle
import pandas as pd

def train():
    X,y=preProcessTrain()
    cv=CountVectorizer()
    
    X=cv.fit_transform(X).toarray()
    classifier=MultinomialNB()
    classifier.fit(X,y)
    
    pickle.dump(classifier,open("model/model.pkl","wb"))
    
    return("model trained and saved successfuly")
    

    
    

In [124]:
news=pd.read_csv("data/train/news.csv")

In [125]:
news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [115]:
sn

'How women lead differently'

In [116]:
sn=preProcessTest(sn)

In [117]:
sn

'women lead differ'

In [1]:
from com_in_nitin_train.trainAPI import train

In [2]:
train()

'model trained and saved successfuly'

In [3]:
import pickle
model=pickle.load(open("model/model.pkl","rb"))

In [ ]:
model.score()

In [9]:
from com_in_nitin_utils.utils import *

In [14]:
X,y=preProcessTrain()

In [11]:
cv=CountVectorizer()

In [12]:
X=cv.fit_transform(X).toarray()

In [13]:
model.score(X,y)

0.9185477505919495

In [ ]:
a=model.predict()

In [2]:
from com_in_nitin_utils.utils import *

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

def predict(news):
    news=preProcessTest(news)
    cv=CountVectorizer()
    news=cv.fit_transform([news]).toarray()
    
    model=pickle.load(open("model/model.pkl","rb"))
    
    prediction=model.predict(news)
    
    return prediction
    

In [4]:
data=pd.read_csv("data/train/news.csv")

In [5]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
p=predict("Syria, Russia, Mosul: Your Friday Evening Briefing - The New York Times")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 7113 is different from 9)

In [9]:
model=pickle.load(open("model/model.pkl","rb"))

In [36]:
news=data['title'][0]

In [37]:
news

'You Can Smell Hillary’s Fear'

In [38]:
news=preProcessTest(news)

In [39]:
news

'smell hillari fear'

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
cv=CountVectorizer()

In [44]:
news

'smell hillari fear'

In [47]:
news=cv.fit_transform([news]).toarray()

In [48]:
predict=model.predict(news)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 7113 is different from 3)

In [49]:
n=pd.read_csv("Anuj/kaggle_fake_train.csv")

In [50]:
n.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [52]:
def fake_news(sample_news):
  sample_news = re.sub(pattern='[^a-zA-Z]',repl=' ', string=sample_news)
  sample_news = sample_news.lower()
  sample_news_words = sample_news.split()
  sample_news_words = [word for word in sample_news_words if not word in set(stopwords.words('english'))]
  ps = PorterStemmer()
  final_news = [ps.stem(word) for word in sample_news_words]
  final_news = ' '.join(final_news)

  temp = cv.transform([final_news]).toarray()
  model=pickle.load(open("model/model.pkl","rb"))
  return model.predict(temp)

In [53]:
n=data['title'][2]

In [54]:
n

'Kerry to go to Paris in gesture of sympathy'

In [55]:
pred=fake_news(n)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 7113 is different from 3)

In [1]:
from com_in_nitin_train.trainAPI import train

In [3]:
train()

'model trained and saved successfuly'

In [4]:
import pandas as pd

In [5]:
data=pd.read_csv("data/train/news.csv")

In [6]:
n=data['title'][0]

In [7]:
from com_in_nitin_utils.utils import preProcessTest

In [8]:
n

'You Can Smell Hillary’s Fear'

In [9]:
n=preProcessTest(n)

In [10]:
n

'smell hillari fear'

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
cv=CountVectorizer(max_features=5000, ngram_range=(1,3))

In [16]:
n=cv.fit_transform([n]).toarray()

In [17]:
import pickle

In [18]:
model=pickle.load(open("model/model.pkl","rb"))

In [19]:
pred=model.predict(n)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5000 is different from 6)

In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from com_in_nitin_utils.utils import *
from com_in_nitin_train.trainAPI import *

In [3]:
X,y=preProcessTrain()

In [4]:
X[0:10]

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter',
 'tehran usa',
 'girl horrifi watch boyfriend left facetim',
 'britain schindler die 106',
 'fact check trump clinton command chief forum',
 'iran reportedli make new push uranium concess nuclear talk']

In [5]:
x=X

In [6]:
cv=CountVectorizer()

In [7]:
x=cv.fit_transform(x).toarray()

In [8]:
x.shape

(6335, 7113)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=0)

In [11]:
lr=LogisticRegression()

In [12]:
lr.fit(x_train,y_train)

LogisticRegression()

In [13]:
lr.score(x_train,y_train)

0.9583662194159431

In [14]:
y_pred=lr.predict(x_test)

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
accuracy_score(y_test,y_pred)

0.823993685872139

In [17]:
news=X[0]

In [18]:
news

'smell hillari fear'

In [19]:
news=cv.transform([news]).toarray()

In [20]:
lr.predict(news)

array(['FAKE'], dtype=object)

In [21]:
data=pd.read_csv("data/train/news.csv")

In [24]:
news=data['title'][10]

In [25]:
news

'With all three Clintons in Iowa, a glimpse at the fire that has eluded Hillary Clinton’s campaign'

In [26]:
from com_in_nitin_utils.utils import *

In [27]:
news=preProcessTest(news)

In [28]:
news

'three clinton iowa glimps fire elud hillari clinton campaign'

In [40]:
news=cv.transform([news]).toarray()

In [30]:
lr.predict(news)

array(['REAL'], dtype=object)

In [31]:
import pickle

In [33]:
pickle.dump(lr,open("lr.pkl","wb"))

In [34]:
model=pickle.load(open("lr.pkl","rb"))

In [35]:
model.predict(news)

array(['REAL'], dtype=object)

In [24]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

In [25]:
data['news']=data['title']+data['text']

In [27]:
data=data.drop(['Unnamed: 0','title','text'],axis=1)

In [28]:
data.head()

,label,news
0,FAKE,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,FAKE,Watch The Exact Moment Paul Ryan Committed Pol...
2,REAL,Kerry to go to Paris in gesture of sympathyU.S...
3,FAKE,Bernie supporters on Twitter erupt in anger ag...
4,REAL,The Battle of New York: Why This Primary Matte...


In [1]:
from com_in_nitin_train.trainAPI import train
from com_in_nitin_predict.predictAPI import predict

In [2]:
train()

'model trained and saved successfuly'

In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("data/train/news.csv")

In [4]:
news=data['title'][0]+data['text'][0]

In [5]:
news

'You Can Smell Hillary’s FearDaniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is

In [6]:
predict(news)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5000 is different from 406)

In [8]:
from com_in_nitin_utils.utils import preProcessTest

In [36]:
from com_in_nitin_predict.predictAPI import predict

In [37]:
news=data['title'][25]

In [38]:
news

'Anti-Trump forces seek last-ditch delegate revolt'

In [39]:
predict(news)

NotFittedError: Vocabulary not fitted or provided